In [1]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
import dask.config
from distributed import LocalCluster, Client

In [2]:
# dask.config.config['distributed']['comm']['timeouts']['connect'] = '300s'
# dask.config.config['distributed']['scheduler']['allowed-failures'] = 12
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.3, 'spill': 0.5, 'pause': 0.7, 'terminate': 0.95}

In [ ]:
n_workers=2
cluster = LocalCluster(n_workers=n_workers, threads_per_worker=6, memory_limit=f'{20E9/n_workers}')#silence_logs=logging.ERROR)
client = Client(cluster)
client

In [4]:
df = dd.read_parquet('../data/small_partitions_geohashed_gps_pts', engine='pyarrow')
# df = df.partitions[1217:].repartition(npartitions=8).persist()
df.head()

,latitude,longitude,geohash
index,,,
0,-90.0,-39.374215,50h000084212
1,-90.0,-5.624215,5bh000084212
2,-90.0,64.688241,j2n000081208
3,-90.0,90.000000,n00000000000
4,-90.0,98.438280,n0n0000840j8


In [7]:
%%time 
t0 = time.time()
usdf = df[df.latitude.between(24.396308, 49.384358) & df.longitude.between(-124.848974, -66.885444)]
usdf.to_parquet('../data/contiguous_us_w_geohash.parquet', engine='pyarrow', compression=None)
dt_hr = (time.time() - t0)/60/60

CPU times: user 1min 57s, sys: 7.55 s, total: 2min 4s
Wall time: 13min 35s


In [8]:
with open(f'{datetime.now()}_us_filter_time.csv', 'w') as f:
    f.write(f'dt_hr, {dt_hr}')

In [10]:
usdf = dd.read_parquet('../data/coto_parquetus_us_w_geohash.parquet', engine='pyarrow', compression=None)

In [26]:
%%time
usdf.repartition(npartitions=200).to_parquet('../data/contiguous_us_w_geohash.parquet', engine='pyarrow', compression=None)

CPU times: user 7.21 s, sys: 590 ms, total: 7.8 s
Wall time: 45.1 s


In [28]:
usdf = dd.read_parquet('../data/contiguous_us_w_geohash.parquet', engine='pyarrow', compression=None)